|I/O-Bound Process|	CPU-Bound Process|
|--|--|
| Your program spends most of its time talking to a slow device, like a network connection, a hard drive, or a printer.|	You program spends most of its time doing CPU operations.|
| Speeding it up involves overlapping the times spent waiting for these devices.| Speeding it up involves finding ways to do more computations in the same amount of time.|

| Concurrency Type | Switching Decision | Number of Processors |
|--------------|-----------|------------|
| Pre-emptive multitasking (threading) | The operating system decides when to switch tasks external to Python.| 1|
| Cooperative multitasking (asyncio) | The tasks decide when to give up control. | 1|
| Multiprocessing (multiprocessing) | The processes all run at the same time on different processors.| Many|

In [1]:
import threading
import multiprocessing
import time
import sys

In [2]:
def cpu_func(result, niters):
    '''
    A useless CPU bound function.
    '''
    for i in range(niters):
        result = (result * result * i + 2 * result * i * i + 3) % 10000000
    return result

class CpuThread(threading.Thread):
    def __init__(self, niters):
        super().__init__()
        self.niters = niters
        self.result = 1
    def run(self):
        self.result = cpu_func(self.result, self.niters)

class CpuProcess(multiprocessing.Process):
    def __init__(self, niters):
        super().__init__()
        self.niters = niters
        self.result = 1
    def run(self):
        self.result = cpu_func(self.result, self.niters)

class IoThread(threading.Thread):
    def __init__(self, sleep):
        super().__init__()
        self.sleep = sleep
        self.result = self.sleep
    def run(self):
        time.sleep(self.sleep)

class IoProcess(multiprocessing.Process):
    def __init__(self, sleep):
        super().__init__()
        self.sleep = sleep
        self.result = self.sleep
    def run(self):
        time.sleep(self.sleep)

In [14]:
if __name__ == '__main__':
    cpu_n_iters = 500000
    sleep = 1
    cpu_count = multiprocessing.cpu_count()
    input_params = [
        (CpuThread, cpu_n_iters),
        (CpuProcess, cpu_n_iters),
        (IoThread, sleep),
        (IoProcess, sleep),
    ]
    header = ['nthreads']
    results_list = []
    for thread_class, _ in input_params:
        header.append(thread_class.__name__)
    print(' '.join(header))
    for nthreads in range(1, 2 * cpu_count):
        results = [nthreads]
        for thread_class, work_size in input_params:
            start_time = time.time()
            threads = []
            for i in range(nthreads):
                thread = thread_class(work_size)
                threads.append(thread)
                thread.start()
            for i, thread in enumerate(threads):
                thread.join()
            results.append(time.time() - start_time)
        print(' '.join('{:.6e}'.format(result) for result in results))
        results_list.append(results)

nthreads CpuThread CpuProcess IoThread IoProcess
1.000000e+00 3.619986e-01 1.433006e+00 1.003213e+00 1.417256e+00
2.000000e+00 6.270070e-01 1.449982e+00 1.003901e+00 1.492973e+00
3.000000e+00 9.260094e-01 1.492968e+00 1.007475e+00 1.496097e+00
4.000000e+00 1.302943e+00 1.507056e+00 1.009218e+00 1.525013e+00
5.000000e+00 1.561030e+00 1.581995e+00 1.010297e+00 1.591685e+00
6.000000e+00 1.864017e+00 1.655797e+00 1.009461e+00 1.655607e+00
7.000000e+00 2.313000e+00 1.785013e+00 1.011499e+00 1.745467e+00
8.000000e+00 2.557989e+00 1.811211e+00 1.013873e+00 1.801916e+00
9.000000e+00 2.909030e+00 1.929888e+00 1.015188e+00 1.909618e+00
1.000000e+01 3.192922e+00 2.018377e+00 1.016929e+00 2.017855e+00
1.100000e+01 3.489019e+00 2.115698e+00 1.018779e+00 2.124717e+00
1.200000e+01 3.771013e+00 2.227961e+00 1.017799e+00 2.229400e+00
1.300000e+01 4.027010e+00 2.377691e+00 1.022096e+00 2.510583e+00
1.400000e+01 4.358999e+00 2.463514e+00 1.020988e+00 2.505461e+00
1.500000e+01 4.697000e+00 2.534052e+00 1.